In [81]:
import PyPDF2
import os
import re 

In [82]:
# bookmark page + VA = pdf page
VA = 2

In [83]:
pdfiles = os.listdir('pdf')
for pdfile in pdfiles:
    pdf_path = os.path.join('pdf',pdfile)
    pdfname, _ = os.path.splitext(pdf_path)
    mark_path = os.path.join('mark',pdfname+'.txt')
    output_path = os.path.join('marked_pdf',pdfile)

    if not mark_path.exists():
        continue

    # 打开bookmark文件
    with open(mark_path,'r') as f:
        #读取文件中的每一行
        lines = f.readlines()
    # 去除每行结尾的换行符
    lines = [line.strip() for line in lines if line.strip()]

    chapter_pattern = r'^(第+\d+章+\s+\S+)\s+(\d+)$'
    part1_pattern = r'^(\d+\.\d+\s+\S+)\s+(\d+)$'
    part2_pattern = r'^(\d+\.\d+\.\d+\s+\S+)\s+(\d+)$'
    ## pattern is r'^(title) (page)'

    # 打开PDF文件并读取
    with open(pdf_path, 'rb') as input_file:
        reader = PyPDF2.PdfReader(input_file)
        # 创建PDF写入器并打开输出文件
        writer = PyPDF2.PdfWriter()

        with open(output_path, 'wb') as output_file:
            chapter_bookmark = None
            part1_bookmark = None
            errors = []
            # 将PDF页面复制到输出文件
            for i in range(len(reader.pages)):
                writer.add_page(reader.pages[i])
            # 将书签添加到输出文件的目录中
            # 匹配bookmark
            for line in lines: 
                chapter_match = re.search(chapter_pattern, line)
                part1_match = re.search(part1_pattern, line)
                part2_match = re.search(part2_pattern, line)
                
                # 为PDF添加书签
                if chapter_match:
                    title, page = chapter_match.group(1) , int(chapter_match.group(2))
                    chapter_bookmark = writer.add_outline_item(title, page+VA)
                elif part1_match:
                    title, page = part1_match.group(1) , int(part1_match.group(2))
                    part1_bookmark = writer.add_outline_item(title, page+VA, parent=chapter_bookmark)
                elif part2_match:
                    title, page = part2_match.group(1) , int(part2_match.group(2))
                    part2_bookmark = writer.add_outline_item(title, page+VA, parent=part1_bookmark)
                else:
                    errors.append(line)
            if errors != []:
                for error in errors:
                    print(error)
                raise Exception(f"Cannot be recognized")
            # 将PDF写入到输出文件中
            writer.write(output_file)
